In [1]:
!pip install transformers

In [2]:
!pip install SpeechRecognition

In [3]:
!pip install PyAudio

In [4]:
import torch
import speech_recognition as sr
from transformers import BertTokenizer, BertForQuestionAnswering, MarianMTModel, MarianTokenizer

C:\Users\prana\AppData\Local\Programs\Python\Python310\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [5]:
with open("dataset.txt", "r") as f:
    context = f.read()

In [6]:
recognizer = sr.Recognizer()

In [7]:
translator = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
translator_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

C:\Users\prana\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [9]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt")
    start_positions, end_positions = model(**inputs).start_logits.argmax(), model(**inputs).end_logits.argmax()
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_positions:end_positions+1]))
    return answer

In [10]:
def translate_to_hindi(text):
    inputs = translator_tokenizer.encode(text, return_tensors="pt")
    translated = translator.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    translation = translator_tokenizer.decode(translated[0], skip_special_tokens=True)
    return translation

In [11]:
# QnA in English 

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break
    else:
        answer = answer_question(user_input, context)
        print("Chatbot: ", answer)
        

You: Who is HOD of college ?
Chatbot:  dr . lata ragha
You: Who is Dr. Lata Ragha ?
Chatbot:  hod of college
You: what is address of college ?
Chatbot:  agnel technical education complex , sector 9 - a , vashi , navi mumbai , maharashtra , india , pin - 400703
You: When is college conducting session on career counselling ?
Chatbot:  20th may 2023
You: exit
Chatbot: Goodbye!


In [12]:
# QnA in English and Hindi translation of answer

while True:
    user_input = input("You (in English): ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break
    else:
        hindi_question = translate_to_hindi(user_input)
        answer = answer_question(user_input, context)
        hindi_answer = translate_to_hindi(answer)
        print("Chatbot (in English): ", answer)
        print("Chatbot (in Hindi): ", hindi_answer)

You (in English): what is Fr. CRIT ?
Chatbot (in English):  a leading engineering college in mumbai university
Chatbot (in Hindi):  मांबा विश्‍वविद्यालय में एक प्रमुख इंजीनियरिंग कॉलेज
You (in English): what are career counseling sessions 
Chatbot (in English):  career counseling sessions can be a valuable tool for anyone looking to make informed decisions about their career path
Chatbot (in Hindi):  जो कोई अपने करियर के बारे में जानकारी हासिल करने के लिए ज़रूरी फैसले लेता है, उसके लिए स्कूल की पढ़ाई बहुत ज़रूरी हो सकती है
You (in English): exit
Chatbot: Goodbye!


In [13]:
# Voice command input via microphone

while True:
    try:
        with sr.Microphone() as source:
            print("Speak something...")
            audio = recognizer.listen(source)
            print("Audio captured, processing...")
            
        user_input = recognizer.recognize_google(audio)
        print("Input: ", user_input)
        
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break
        else:
            answer = answer_question(user_input, context)
            print("Chatbot: ", answer)

    except sr.UnknownValueError:
        print("Sorry, I couldn't understand what you said.")
    except sr.RequestError as e:
        print(f"Could not request results from Google Web Speech API; {e}")

Speak something...
Audio captured, processing...
Input:  who is HOD of college
Chatbot:  dr . lata ragha
Speak something...
Audio captured, processing...
Sorry, I couldn't understand what you said.
Speak something...
Audio captured, processing...
Input:  exit
Chatbot: Goodbye!
